In [1]:
import os
import cv2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

IMG_DIM = 364

In [2]:
def bensYCC(bgr, weight=4, gamma=10):
        
    # convert to y, cr, cb so that we can modify the image based on just the y (brightness)
    ycc = cv2.cvtColor(bgr, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycc)

    # perform bens algorithm on the y component
    y = cv2.addWeighted(y, weight, cv2.GaussianBlur(y, (0,0), gamma), -weight, 128)

    # merge the ycc back together, and recolor it
    ycc_modified = cv2.merge((y, cr, cb))
    bens = cv2.cvtColor(ycc_modified, cv2.COLOR_YCrCb2BGR)
    
    return bens 


def bensGray(gray, weight=4, gamma=10):
        
    # perform bens algorithm on the y component
    bens = cv2.addWeighted(gray, weight, cv2.GaussianBlur(gray, (0,0), gamma), -weight, 128)
    
    return bens 

In [3]:
def crop(gray, img, percent_smaller):
    
    thresh = 8
    
    top    = 0
    left   = 0
    bottom = gray.shape[0] - 1
    right  = gray.shape[1] - 1
    
    # work in from the top and bottom along the middle collumn
    middleCol = gray[:, int(gray.shape[1]/2)] > thresh
    while middleCol[top] == 0:
        top += 1
    while middleCol[bottom] == 0:
        bottom -= 1
        
    # work in from the sides along the middle row
    middleRow = gray[int(gray.shape[0]/2)] > thresh
    while middleRow[left] == 0:
        left += 1
    while middleRow[right] == 0:
        right -= 1
        
    height = bottom - top
    width  = right - left
    
    bottom -= int(percent_smaller*height)
    top    += int(percent_smaller*height)
    right  -= int(percent_smaller*width)
    left   += int(percent_smaller*width)
        
    if height < 100 or width < 100:
        print("Error: squareUp: bottom:", bottom, "top:", top)
        print("Error: squareUp: right:", right, "left:", left)
        return img
    
    return img[top:bottom, left:right]

    

In [4]:
def reflectAndSquareUp(img):
    
    height = img.shape[0]
    width  = img.shape[1]
    
    # if its portrait mode, it's probably already kind of square. Make it properly square by cutting
    # down the height until the dimensions match
    if (height > width):
        
        offset = int((height - width)/2)
        return img[offset:offset+width]
    
    # otherwise, do the whole reflection thingo
    else:
        if len(img.shape) == 3:
            new_img = np.zeros((width, width, img.shape[2]), np.uint8)
        else:
            new_img = np.zeros((width, width), np.uint8)

        #  0  |
        #     |
        #  h1 |####
        #     |####
        #     |####
        #  h2 |
        #     |

        h1 = int((width - height)/2)
        h2 = h1 + height

        # paste the original into the center
        new_img[h1:h2,:] = img

        # paste in the reflections
        for i in range(h1):
            new_img[h1-i] = img[i]

        for i in range(width - h2):
            new_img[h2+i] = img[height - i - 1]

        return new_img

def circleMask(img):
    
    if (img.shape[0] != img.shape[1]):
        print("Error: circle mask assumes square image")
        return img
    
    dim = img.shape[0]
    half = int(dim/2)
    
    # crop out circle:
    circle_mask = np.zeros((dim, dim), np.uint8)
    circle_mask = cv2.circle(circle_mask, (half, half), half, 1, thickness=-1)

    return cv2.bitwise_and(img, img, mask=circle_mask)

def clahe_gray(gray, clipLimit=4.0, grid=8):

    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=(grid,grid))
    return clahe.apply(gray)

def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                     for i in np.arange(0, 256)]).astype("uint8")

    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)




In [ ]:
# # Test different versions of colourful eyes

# # the main dataframes
# df_2019 = pd.read_csv(f"../aptos2019/train.csv") 
# df_2019.id_code = df_2019.id_code.apply(lambda x: x + ".png")

# # df_2015 = pd.read_csv(f"../aptos2015/trainLabels.csv") 
# # df_2015.image   = df_2015.image.apply(lambda   x: x + ".jpeg")

# # Display some random images from Data Set with class categories.
# figure=plt.figure(figsize=(22,20))
# for target_class in range(5):
#     for i, (idx, row ) in enumerate(df_2019.loc[df_2019.diagnosis == target_class]
#                                     .sample(1)
#                                     .iterrows()):
#         # open the file
#         imagefile = f"../aptos2019/train_images/{row['id_code']}" 
#         img = cv2.imread(imagefile)
#         green = img[:,:,1]
        
#         # original version
#         rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         ax = figure.add_subplot(5,4, int(target_class)*4+2*i+1)
#         plt.imshow(rgb)
        
        
#         # colour ben
#         cropped = crop(green, img, 0.02)
#         squared = reflectAndSquareUp(cropped)
#         resized = cv2.resize(squared, (IMG_DIM, IMG_DIM))
#         circled = circleMask(resized)
#         equalised = adjust_gamma(circled, 1+np.log(100)-np.log(np.median(circled)))
        
#         ax = figure.add_subplot(5,4, int(target_class)*4+2*i+2)
#         plt.imshow(cv2.cvtColor(bensYCC(equalised), cv2.COLOR_BGR2RGB))
        
#         # ------------ greys --------------
#         cropped = crop(green, green, 0.02)
#         squared = reflectAndSquareUp(cropped)
#         resized = cv2.resize(squared, (IMG_DIM, IMG_DIM))
#         circled = circleMask(resized)
#         equalised = adjust_gamma(circled, 1+np.log(100)-np.log(np.median(circled)))
        
#         # green ben
#         ax = figure.add_subplot(5,4, int(target_class)*4+2*i+3)
#         plt.imshow(bensGray(equalised), cmap='gray')
        
# #         green clahe
#         ax = figure.add_subplot(5,4, int(target_class)*4+2*i+4)
#         plt.imshow(clahe_gray(equalised), cmap='gray')
        
imagefile = f"../aptos2015/train_images/1986_left.jpeg" 
img = cv2.imread(imagefile)
plt.imshow(img)


In [ ]:
def process_images(year, test=True):
    
    # set the year
    if year == "2019":
        directory = "../aptos2019/train_images/" 
        file_end  = ".png"
        df        = pd.read_csv(f"../aptos2019/train.csv") 
    elif year == "2015":
        directory = "../aptos2015/train_images/" 
        file_end  = ".jpeg"
        df        = pd.read_csv(f"../aptos2015/trainLabels.csv") 
        df["id_code"] = df.image
        
    df.id_code = df.id_code.apply(lambda x: x + file_end)
    
    # go through all zeh images und changen them
    if test:
        figure=plt.figure(figsize=(22,20))
    for count, file_name in enumerate(df.id_code.values):

        if count % 100 == 0:
            print(count)
        
        bens_colour_file = directory + "ben_colour_" + file_name
        bens_green_file = directory + "ben_green_" + file_name
        clahe_green_file = directory + "clahe_green_" + file_name
        
        if os.path.isfile(bens_colour_file) and \
            os.path.isfile(bens_green_file) and \
            os.path.isfile(clahe_green_file):
                continue
                
        # ---- do preprocessing common to all ----
        try:
            bgr = cv2.imread(directory + file_name)
            if test:
                rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
                ax = figure.add_subplot(5,4, 4*count+1)
                plt.imshow(rgb)

            green   = bgr[:,:,1] # use green as a greyscale
            cropped = crop(green, bgr, 0.02)
            squared = reflectAndSquareUp(cropped)

            resized_bgr = cv2.resize(squared, (IMG_DIM, IMG_DIM))
            circled_bgr = circleMask(resized_bgr)
            equalised_bgr = adjust_gamma(circled_bgr, 1+np.log(100)-np.log(np.median(circled_bgr)))

            squared_green = squared[:,:,1]
            resized_green = cv2.resize(squared_green, (IMG_DIM, IMG_DIM))
            circled_green = circleMask(resized_green)
            equalised_green = adjust_gamma(circled_green, 1+np.log(100)-np.log(np.median(circled_green)))

            # ----- Bens Coloured -----
            if not os.path.isfile(bens_colour_file):
                new_img = bensYCC(equalised_bgr)
                if test:
                    rgb = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
                    ax = figure.add_subplot(5,4, 4*count+2)
                    plt.imshow(rgb)
                else:
                    cv2.imwrite(bens_colour_file, new_img)

            # ----- Bens Green -----
            if not os.path.isfile(bens_green_file):
                new_img = bensGray(equalised_green)
                if test:
                    ax = figure.add_subplot(5,4, 4*count+3)
                    plt.imshow(new_img, cmap='gray')
                else:
                    cv2.imwrite(bens_green_file, new_img)

            # ----- Clahe Green -----
            if not os.path.isfile(clahe_green_file):
                new_img = clahe_gray(equalised_green)
                if test:
                    ax = figure.add_subplot(5,4, 4*count+4)
                    plt.imshow(new_img, cmap='gray')
                else:
                    cv2.imwrite(clahe_green_file, new_img)
        except:
            print(f"Error: {file_name} failed")
            
        if test and count == 4:
            break


process_images("2015", test=False)

0
100
200
300
400
Error: 492_right.jpeg failed
500
600
700
800
900
1000
1100
1200
1300
1400
1500
Error: 1986_left.jpeg failed
1600
1700
Error: 2157_left.jpeg failed
Error: 2236_right.jpeg failed
1800
Error: 2248_left.jpeg failed
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:50: RuntimeWarning: divide by zero encountered in log


10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
Error: squareUp: bottom: 1943 top: 1924
Error: squareUp: right: 2337 left: 2319
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700
22800
22900
23000
23100
23200
23300
23400
23500
23600
